# Importing Packages


In [144]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as ureq
import requests
import urllib
import re
import os

data = pd.read_excel("Input.xlsx")

In [145]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [146]:
data['URL']

0     https://insights.blackcoffer.com/rising-it-cit...
1     https://insights.blackcoffer.com/rising-it-cit...
2     https://insights.blackcoffer.com/internet-dema...
3     https://insights.blackcoffer.com/rise-of-cyber...
4     https://insights.blackcoffer.com/ott-platform-...
                            ...                        
95    https://insights.blackcoffer.com/what-is-the-r...
96    https://insights.blackcoffer.com/impact-of-cov...
97    https://insights.blackcoffer.com/contribution-...
98    https://insights.blackcoffer.com/how-covid-19-...
99    https://insights.blackcoffer.com/how-will-covi...
Name: URL, Length: 100, dtype: object

In [147]:
article_titles = []
for i, url in enumerate(data['URL']):
    response_code = requests.get(url)
    soup = bs(response_code.text, 'html.parser')
    article_title = soup.find('title').text
    article_titles.append(article_title)


In [148]:
article_titles

['Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040. - Blackcoffer Insights',
 'Rising IT Cities and Their Impact on the Economy, Environment, Infrastructure, and City Life in Future - Blackcoffer Insights',
 "Internet Demand's Evolution, Communication Impact, and 2035's Alternative Pathways - Blackcoffer Insights",
 'Rise of Cybercrime and its Effect in upcoming Future - Blackcoffer Insights',
 'OTT platform and its impact on the entertainment industry in Future. - Blackcoffer Insights',
 'The rise of the OTT platform and its impact on the entertainment industry by 2040. - Blackcoffer Insights',
 'Rise of Cyber Crime and its Effects - Blackcoffer Insights',
 'Rise of Internet Demand and Its Impact on Communications and Alternatives by the Year 2035 - Blackcoffer Insights',
 'Rise of Cybercrime and its Effect by the Year 2040. - Blackcoffer Insights',
 'Rise of Cybercrime and its Effect by the Year 2040. - Blackcoffer Insights',

In [149]:
url = 'https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040/'
response_code = requests.get(url)
soup = bs(response_code.text, 'html.parser')
all_texts = soup.find("div",class_="td-post-content tagdiv-type").get_text(strip = True, separator='\n')
firstdata = all_texts.splitlines()
firstdata

['In the present world, there is an increase in IT cities,\xa0 which is the demand of all the people who are living with\xa0 #frolic. \xa02040 isn’t that far into the future. There won’t be any revolutionary design change in the exteriors of the cities, that is how they look like from the #outside. The building and infrastructure will be similar to today.',
 'According to one of the most frequently cited definitions of a smart city (Albino et al., 2015), “a city is smart when investments in human and social capital, as well as traditional (transport) and modern (ICT) communication infrastructure, fuel sustainable economic growth and a high quality of life, with wise management of natural resources, through participatory governance” (Caragliu et al., 2011, p. 70). Given that cities can be viewed as integrated systems, traditional infrastructure may include vertical systems other than transportation, such as energy, governance, and urban structures. “Smartness” then emerges as a set of l

In [150]:
with open('/content/merge_from_ofoct.txt', 'r', encoding='latin1') as stopwordsfile:
    # Read the contents of the file
    stopwords = stopwordsfile.read().lower()
    stopWordList = stopwords.split('\n')
    stopWordList


In [151]:
with open('/content/positive-words.txt', 'r', encoding='latin1') as poswordsfile:
    # Read the contents of the file
    poswords = poswordsfile.read().lower()
positiveWordList=poswords.split('\n')

In [152]:
with open('/content/negative-words.txt', 'r', encoding='latin1') as negwordsfile:
    # Read the contents of the file
    negwords = negwordsfile.read().lower()
negativeWordList=negwords.split('\n')

# Cleaning using Stop Words Lists


In [153]:
import nltk
nltk.download('punkt')
global tokens
def tokenizer(data_list):
    # Initialize an empty list to store filtered tokens from all strings
    all_filtered_words = []

    # Tokenize each string in the list
    for text in data_list:
        # Tokenize the current string
        tokens = nltk.word_tokenize(text.lower())

        # Filter tokens using stop words list
        filtered_words = [token for token in tokens if token not in stopWordList]

        # Remove punctuation
        filtered_words = [word for word in filtered_words if word.isalnum()]

        # Append filtered words to the list of all filtered words
        all_filtered_words.extend(filtered_words)

    return all_filtered_words


filtered_tokens = tokenizer(firstdata)
print(filtered_tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['present', 'world', 'increase', 'cities', 'demand', 'people', 'living', 'frolic', '2040', 'isn', 'future', 'won', 'revolutionary', 'design', 'change', 'exteriors', 'cities', 'building', 'infrastructure', 'similar', 'today', 'frequently', 'cited', 'definitions', 'albino', '2015', 'investments', 'human', 'social', 'capital', 'traditional', 'transport', 'modern', 'ict', 'communication', 'infrastructure', 'fuel', 'sustainable', 'economic', 'growth', 'quality', 'life', 'management', 'natural', 'resources', 'participatory', 'governance', 'caragliu', '2011', '70', 'cities', 'viewed', 'integrated', 'systems', 'traditional', 'infrastructure', 'include', 'vertical', 'systems', 'transportation', 'energy', 'governance', 'structures', 'smartness', 'emerges', 'set', 'interconnections', 'technology', 'ict', 'connect', 'critical', 'environment', 'dwellers', 'require', 'drivers', 'trends', 'potential', 'impacts', 'future', 'developments', 'primarily', 'related', 'growing', 'challenges', 'cities', 'env

#Extracting Derived variables(Positive,Negative,Polaity and Subjective score )




1.   **Positive Score**: This score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary and then adding up all the values

1.  **Negative Score**: This score is calculated by assigning the value of -1 for each word if found in the Negative Dictionary and then adding up all the values. We multiply the score with -1 so that the score is a positive number.
2.   **Polarity Score:** This is the score that determines if a given text is positive or negative in nature. It is calculated by using the formula:
Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Range is from -1 to +1

2.  **Subjectivity Score**: This is the score that determines if a given text is objective or subjective. It is calculated by using the formula:
Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
Range is from 0 to +1




In [154]:
def Positive_score(filtered_tokens):
    posword = 0
    for token in filtered_tokens:
        if token in positiveWordList:
            posword += 1
    return posword


In [155]:
def Negative_score(data_list):
    negword = 0
    for token in data_list:
        if token in negativeWordList:
            negword += 1
    return negword


In [156]:
def Polarity_score (Positive_score , Negative_score) :
  return (Positive_score - Negative_score) / ((Positive_score + Negative_score) + 0.000001)

In [157]:
def Subjective_score(Positive_score, Negative_score, total_words):
    return (Positive_score + Negative_score) / (total_words + 0.000001)


# Analysis of Readability



1.   **Average Sentence Length** = the number of words / the number of sentences

2.   **Percentage of Complex words** = the number of complex words / the number of words

3.  **Fog Index** = 0.4 * (Average Sentence Length + Percentage of Complex words)






In [158]:
from nltk.tokenize import sent_tokenize, word_tokenize

def AverageSentenceLength(data_list):

    data_list = ' '.join(data_list)
    Wordcount = len(word_tokenize(data_list))
    SentenceCount = len(sent_tokenize(data_list))
    if SentenceCount > 0:
        Average_Sentence_Length = Wordcount / SentenceCount
    else:
        Average_Sentence_Length = 0  # Handle the case where there are no sentence

    return round(Average_Sentence_Length)

# Combine the lines into a single string


# Example usage:
AverageSentenceLength(firstdata)

26

In [159]:
def complex_word_percentage(data_list):
    data_list = ' '.join(data_list)
    # Assuming tokenizer is defined elsewhere to tokenize the data_list
    tokens = tokenizer(data_list)
    complexWord = 0

    for word in tokens:
        vowels = sum(1 for char in word if char.lower() in 'aeiou')  # Count vowels
        if not word.endswith(('es', 'ed')):
            if vowels > 2:
                complexWord += 1
        ComplexWordPercentage = complexWord/len(tokens)
    return ComplexWordPercentage  # Corrected variable name to return the count of complex words


complex_word_percentage(firstdata)

0.0

In [160]:
def fog_index(Average_Sentence_Length, complex_word_percentage):
    fogIndex = 0.4 * (Average_Sentence_Length + complex_word_percentage)
    return fogIndex

# Average Number of Words Per Sentence


**Average Number of Words Per Sentence** = the total number of words / the total number of sentences



In [161]:
def average_words_per_sentence(text):
    text = ' '.join(text)
    # Split the text into sentences using simple splitting based on common punctuation marks
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]  # Split on period (.), assuming sentences end with period

    # Calculate the total number of words in the entire text
    words = text.split()  # Split text into words based on whitespace
    total_words = len(words)

    # Calculate the total number of sentences
    total_sentences = len(sentences)

    # Calculate the average number of words per sentence
    if total_sentences > 0:
        average_words_per_sentence = total_words / total_sentences
    else:
        average_words_per_sentence = 0  # Handle division by zero if there are no sentences

    return average_words_per_sentence


avg_words_per_sentence = average_words_per_sentence(firstdata)
print("Average words per sentence:", avg_words_per_sentence)


Average words per sentence: 19.303370786516854


# Complex Word Count

Complex words are words in the text that contain more than two syllables.

In [162]:
def complex_word_count(data_list):
    # Assuming tokenizer is defined elsewhere to tokenize the data_list
    tokens = tokenizer(data_list)
    complexWord = 0

    for word in tokens:
        vowels = sum(1 for char in word if char.lower() in 'aeiou')  # Count vowels
        if vowels > 2:
          complexWord += 1
    return complexWord  # Corrected variable name to return the count of complex words


complex_word_count(firstdata)

457

# Word Count
counting total cleaned words present in the text by
removing the stop words (using stopwords class of nltk package).
removing any punctuations like ? ! , . from the word before counting.


In [163]:
def filteredwords_count(data_list):
  all_filtered_words = []

  for text in data_list:
    # Tokenize the current string
    tokens = nltk.word_tokenize(text.lower())

    # Filter tokens using stop words list
    filtered_words = [token for token in tokens if token not in stopWordList]

    # Remove punctuation
    filtered_words = [word for word in filtered_words if word.isalnum()]

    # Append filtered words to the list of all filtered words
    all_filtered_words.extend(filtered_words)
    filteredwordcount = len(all_filtered_words)

  return filteredwordcount

filteredwords_count(firstdata)

768

# Personal Pronouns
To calculate Personal Pronouns mentioned in the text, we use regex to find the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken so that the country name US is not included in the list.

In [164]:
def count_personal_pronouns(text):
    text = ' '.join(text)
    # Convert the text to lowercase for case-insensitive matching
    text_lower = text.lower()

    # Define the list of personal pronouns
    personal_pronouns = ["i", "we", "my", "ours", "us"]

    # Initialize counts dictionary
    counts = {pronoun: 0 for pronoun in personal_pronouns}

    # Split the text into words
    words = text_lower.split()

    # Count the occurrences of personal pronouns
    for word in words:
        # Exclude the word "us" if it's part of a country name like "US"
        if word == 'us' and not ('u.s.' in text_lower or 'united states' in text_lower):
            continue

        # Increment the count for the pronoun if it matches any personal pronoun
        if word in personal_pronouns:
            counts[word] += 1
            result = len(counts)

    return result

count_personal_pronouns(firstdata)

5

# Average Word Length
**Average Word Length is calculated by the formula:**
Sum of the total number of characters in each word/Total number of words


In [165]:
def average_word_length(text):
    text = ' '.join(text)
    # Split the text into words
    words = text.split()

    # Calculate the total number of characters in all words
    total_characters = sum(len(word) for word in words)

    # Calculate the total number of words
    total_words = len(words)

    # Calculate the average word length
    if total_words > 0:
        average_length = total_characters / total_words
    else:
        average_length = 0

    return average_length


average_len = average_word_length(firstdata)
print(average_len)


5.511059371362049


In [166]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URLS = data['URL']
corps = []
article_titles=[]

# Filter out URLs that don't result in a 404 error
URLS = [url for url in URLS if requests.head(url).status_code != 404]

for url in URLS:
    try:
        response = requests.get(url, headers={"User-Agent": "XY"})
        response.raise_for_status()  # Raise an exception for other HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        article_title = soup.find('title').text
        article_titles.append(article_title)

        # Get article text
        text = soup.find(attrs={'class': 'td-post-content'}).get_text(strip = True, separator='\n')
        corps.append(text)

    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Create DataFrame
final = pd.DataFrame({'title': article_titles, 'corps': corps})


In [174]:
def tokenizer(text):
    tokens = nltk.word_tokenize(text.lower())
    filtered_words = [token for token in tokens if token not in stopWordList]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return filtered_words

final['filtered_tokens'] = final['corps'].apply(tokenizer)
final['URL'] = URLS
final['Positive_Score'] = final['filtered_tokens'].apply(Positive_score)
final['Negative_Score'] = final['filtered_tokens'].apply(Negative_score)
final['Polarity_score'] =  np.vectorize(Polarity_score)(final['Positive_Score'],final['Negative_Score'])
final['Subjective_Score'] = final.apply(lambda row: Subjective_score(row['Positive_Score'], row['Negative_Score'], len(row['filtered_tokens'])), axis=1)
final['Average_Sentence_Length'] = final['corps'].apply(AverageSentenceLength)
final['Complex_Word_Percentage'] = final['filtered_tokens'].apply(complex_word_percentage)
final['Fog_Index'] = np.vectorize(Polarity_score)(final['Average_Sentence_Length'],final['Complex_Word_Percentage'])
final['Average_Words_Per_Sentence'] = final['corps'].apply(average_words_per_sentence)
final['Complex_Word_Count'] = final['corps'].apply(complex_word_count)
final['Words_Count'] = final['corps'].apply(filteredwords_count)
final['Personal_Pronouns'] = final['corps'].apply(count_personal_pronouns)
final['Average_Word_Length'] = final['filtered_tokens'].apply(average_word_length)

In [177]:
final.drop(['filtered_tokens','corps'],axis = 1)

,title,Positive_Score,Negative_Score,Polarity_score,Subjective_Score,Average_Sentence_Length,Complex_Word_Percentage,Fog_Index,Average_Words_Per_Sentence,Complex_Word_Count,Words_Count,Personal_Pronouns,Average_Word_Length,URL
0,Rising IT cities and its impact on the economy...,26,6,0.625000,0.066667,73,0.381250,0.989609,72.772152,243,66,5,6.843750,https://insights.blackcoffer.com/rising-it-cit...
1,Rising IT Cities and Their Impact on the Econo...,51,31,0.243902,0.121481,100,0.468148,0.990681,100.402439,423,61,5,7.651852,https://insights.blackcoffer.com/rising-it-cit...
2,"Internet Demand's Evolution, Communication Imp...",36,23,0.220339,0.100000,117,0.537288,0.990858,117.350877,394,28,5,8.313559,https://insights.blackcoffer.com/internet-dema...
3,Rise of Cybercrime and its Effect in upcoming ...,35,74,-0.357798,0.186644,124,0.547945,0.991201,124.288462,376,54,5,8.162671,https://insights.blackcoffer.com/rise-of-cyber...
4,OTT platform and its impact on the entertainme...,20,8,0.428571,0.086687,97,0.439628,0.990976,96.775000,180,18,5,7.761610,https://insights.blackcoffer.com/ott-platform-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Due to the COVID-19 the repercussion of the en...,25,54,-0.367089,0.153101,113,0.408915,0.992789,113.264151,274,43,5,7.343023,https://insights.blackcoffer.com/what-is-the-r...
94,Impact of COVID-19 pandemic on office space an...,21,35,-0.250000,0.142857,133,0.367347,0.994491,133.358974,197,12,5,6.785714,https://insights.blackcoffer.com/impact-of-cov...
95,Contribution of handicrafts (Visual Arts & Lit...,5,3,0.250000,0.040404,88,0.454545,0.989722,88.240000,102,32,5,7.196970,https://insights.blackcoffer.com/contribution-...
96,How COVID-19 is impacting payment preferences?...,12,3,0.600000,0.059761,82,0.378486,0.990811,84.771429,109,51,5,6.390438,https://insights.blackcoffer.com/how-covid-19-...


In [178]:
final.to_excel('Output Data Structure.xlsx')